In [1]:
#                                                   LIBRARIES

# Image preprocesssing- processing
import cv2
from PIL import Image
import numpy as np

# Neural Network libraries
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [2]:
num_classes = 3
train_animals_reduced = np.load("train_animals_reduced.npy")
train_labels_reduced = np.load("train_labels_reduced.npy")
test_animals_reduced = np.load("test_animals_reduced.npy")
test_labels_reduced = np.load("test_labels_reduced.npy")

In [3]:
# One hot encoding for labels
train_labels_reduced=keras.utils.to_categorical(train_labels_reduced,num_classes)
test_labels_reduced=keras.utils.to_categorical(test_labels_reduced,num_classes)

In [4]:
# import sequential model and all the required layers
def create_model():
    #make model
    model=Sequential()
    # Pairs of Conv2D layer and MaxPool2D Layer with increasing filter sizes ( 16,32 ,64). This helps to make image grow more in depthwise and become more flatten.
    # Maxpool: great as they optimize the training time

    '''# Pair 1 (16)
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))

    # Pair 2 (32)
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))

    # Pair 3 (64)
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))

    # Dropout layers to reduce overfitting
    model.add(Dropout(0.2))
    model.add(Flatten())


    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))'''

    # capas de calculo -> ir de menor a mayor, suelen ser potencias / multiplos de 2 (las layers)

    model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu",input_shape=(50,50,3))) # parte imp = relu
    model.add(MaxPooling2D(pool_size=2))

    model.add(Conv2D(filters=128,kernel_size=3,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))

    model.add(Conv2D(filters=256,kernel_size=3,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=4))

    model.add(Dropout(0.2))
    model.add(Flatten()) # without flatten output shape = (50, 50, 6) -> flatten = (None, 6) which we need to get layer output


    # capas de clasificacion

    model.add(Dense(200,activation="relu"))

    # Final dense layer with num_classes nodes = categories of animals we have in the set
    # Softmax activation is used to give scores to these categories which lie between 0 and 1.
    model.add(Dense(num_classes, activation="softmax"))
    model.summary()

    # compile the model
    # We use loss function as categorical_crossentropy and Adam optimizer

    # if binary data -> loss = Binary Cross Entropy and activation = sigmoid
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                      metrics=['accuracy'])

    return model

In [5]:
model = create_model()
model.load_weights('model.h5')

loss, acc = model.evaluate(test_animals_reduced, test_labels_reduced, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 50, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 25, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 256)        0